### Web Scraping
 Técnica basada en extraer datos que provengan de la Web, se requieren de librerías específicas para emplearlo, algunas de estas contienen módulos que nos permiten realizar un manejo de excepciones y que debido a los diferentes desafíos e inconvenientes que trae la navegación como usuario en la Web contribuyen para una recolección de datos exitosa.

In [89]:
import pandas as pd
import numpy as np
from selenium import webdriver
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, ElementNotInteractableException
import plotly.express as px
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

#### Extrayendo las ofertas de empleo (Data Science) en Linkedin del último mes

In [ ]:
option = webdriver.ChromeOptions()
option.add_argument("--window-size=1300,700")

driver = webdriver.Chrome(options=option)
    
try:
    driver.get("https://www.linkedin.com")
    job_search = WebDriverWait(driver, 10).until(EC.presence_of_element_located(
        (By.XPATH, '//a[@href="https://www.linkedin.com/jobs/search?trk=guest_homepage-basic_guest_nav_menu_jobs"]'))
    )
    job_search.click()
    
except NoSuchElementException:   
    driver.back()       
    job_search = WebDriverWait(driver, 10).until(EC.presence_of_element_located(
        (By.XPATH, '//a[@href="https://www.linkedin.com/jobs/search?trk=guest_homepage-basic_guest_nav_menu_jobs"]'))
    )
    job_search.click()
    
finally:  
    quit = WebDriverWait(driver, 10).until(EC.presence_of_element_located(
        (By.XPATH, '//button[@aria-label="Descartar"]'))
    )
    quit.click()
    time.sleep(5)
    
    input_workstation = driver.find_element(By.XPATH, '//input[@aria-controls="job-search-bar-keywords-typeahead-list"]')
    input_workstation.send_keys("Data Science")
    input_location = driver.find_element(By.XPATH, '//input[@aria-controls="job-search-bar-location-typeahead-list"]')
    input_location.clear()
    input_location.send_keys("Argentina")
    input_location.send_keys(Keys.RETURN)    
    
    driver.find_element(By.XPATH, '//button[@aria-label="Filtro «Fecha de publicación». Se ha aplicado el filtro «Cualquier momento». Al hacer clic en este botón, se muestran todas las opciones del filtro «Fecha de publicación»."]').click()
    driver.find_element(By.XPATH, '//input[@id="f_TPR-1"]').click()
    driver.find_element(By.CLASS_NAME, 'filter__submit-button').click()
    
    workstation = driver.find_elements(By.CLASS_NAME, 'base-search-card__title')
    company = driver.find_elements(By.CLASS_NAME, 'hidden-nested-link')
    location = driver.find_elements(By.CLASS_NAME, 'job-search-card__location')
    time = driver.find_elements(By.CLASS_NAME, 'job-search-card__listdate')
 
# -------- Una vez obtenidos los datos los organizamos en un Data Frame --------

data = {
    "workstation": [],
    "company": [],
    "location": [],
    "time": []
}

len_col = np.array([len(workstation),len(location),len(company),len(time)])
min_len_col = len_col.min()

def data_org(data, column, element):
    for e in element:
        data[column].append(e.text)
        if len(data[column]) == min_len_col:
            break
    return data

data_org(data, "workstation", workstation)
data_org(data, "company", company)
data_org(data, "location", location)
data_org(data, "time", time)

df = pd.DataFrame(data)
df

### Data Wrangling
Técnica que se emplea para procesar, manipular y transformar variables con el fin de crear información valiosa, bien organizada y estructurada.

In [ ]:
print(df["location"].value_counts())

# Mapeo de datos: dado que muchas ubicaciones son similares y sobreinforman, las reemplazamos y resumimos la iformación

data_mapping = {
    "Ciudad Autónoma de Buenos Aires, Provincia de Buenos Aires, Argentina": "Ciudad Autónoma de Buenos Aires, Argentina",
    "Buenos Aires, Provincia de Buenos Aires, Argentina": "Provincia de Buenos Aires, Argentina",
    "Buenos Aires y alrededores": "Provincia de Buenos Aires, Argentina"
}

df["location"] = df["location"].apply(lambda x : data_mapping.get(x, x))
print("---------------------------------------------------------------------------")
print(df["location"].value_counts())

#### Dashboard que revela la distribución de ubicaciones y fechas de publicación de las ofertas

In [ ]:
app = dash.Dash(__name__)

app.layout = html.Div(id="body",className="e8_body",children=[
    html.H1("Ofertas de trabajo en Linkedin", className="e8_title"),
        dcc.Dropdown(id="dropdown",className="e8_dropdown",
                    options = [
                        {"label":"Ubiación (País, Ciudad, Barrio)","value":"location"},
                        {"label":"Tiempo de publicación","value":"time"},
                    ],
                    value="location",
                    multi=False,
                    clearable=False),
    html.Div(className="e8_div",children=[
        dcc.Graph(id="graph_1",className="e8_graph",figure={}),
        dcc.Graph(id="graph_2",className="e8_graph",figure={})
    ])
])

@app.callback(
    [Output(component_id="graph_1",component_property="figure"),
    Output(component_id="graph_2",component_property="figure")],
    [Input(component_id="dropdown",component_property="value")]
)

def update_graph(slct_var):
    values_count = df[slct_var].value_counts().reset_index()
    barplot = px.bar(values_count, x=slct_var, y="count", title='Cantidades contadas')
    if slct_var == "location":
        barplot.update_layout(xaxis=dict(tickfont=dict(size=7)))
    piechart = px.pie(values_count, values='count', names=slct_var, title='Porcentages de cantidades')
    if slct_var == "location":
        piechart.update_layout(legend=dict(font=dict(size=6)))
    
    return barplot, piechart
    
if __name__ == "__main__":
    app.run_server(debug=False)

#### Observación y opinión
Como podemos observar gran parte de las ofertas no especifican su ubicación, aunque bastantes puesto son de forma remota, para Juniors o Trainees es diferente a estos casos y si me parece necesario por lo menos indicar la comuna en que se encuentran las oficinas de trabajo.